## Закрытие отчетности

### Импорт библиотек

In [2]:
import pandas as pd
from datetime import datetime
from getpass import getpass, getuser
from pathlib import Path
import sys
from importlib import reload
home_path = str(Path.cwd().parent.parent)
cur_path = str(Path.cwd().parent)
sys.path.append(cur_path + r'\0_sys')
import base_functions as bf
reload(bf)
import glob
import polars
import os
import duckdb as ddb
import gc
import gzip

In [ ]:
# параметры duckdb
ddb.execute(""" 
           SET memory_limit = '2GB';
           SET temp_directory = '/tmp/duckdb_swap';
           SET max_temp_directory_size = '3GB';
           """).df()
duckdb.execute("SET GLOBAL pandas_analyze_sample = 100_000") #не использовать, оставил для справки

## Юбилейное

In [2]:
raw_data_path = r'\\*\FolderRedirection-6\*\Desktop\x5_work\projects_files\13_chances\raw_data\mondelez_0412'

* participant

In [42]:
# исходный файл
raw_data_file = raw_data_path + '\\' + 'lk_chance_data_participant.csv'
# схема
file_schema = {'dataflow_m': 'DATE'
              , 'x5id': 'VARCHAR'
              , 'grant_date_m': 'DATE'
              , 'action_id': 'int32'
              , 'nickname': 'VARCHAR'
             }
# запрос
df_participant = ddb.sql(f"""
   SELECT *
   FROM read_csv('{raw_data_file}',
   delim = ';',
   quote = '"',
   header = true,
   columns = {file_schema}
   )
""")

* participant_progress

In [43]:
# исходный файл
raw_data_file = raw_data_path + '\\' + 'lk_chance_data_participant_progress_w_item.csv'
# запрос
df_participant_progress = ddb.sql(f"""
   SELECT 
       progress_id
       , x5id
       , task_id
       , period_id
       , date_trunc('day', item_date)::date as item_date
       , trace_id
   FROM read_csv('{raw_data_file}',
   delim = ';',
   quote = '"',
   header = true
   )
""")

* task

In [44]:
# исходный файл
raw_data_file = raw_data_path + '\\' + 'lk_chance_data_task.csv'
# схема
file_schema = {'dataflow_dt': 'DATE'
              , 'task_id': 'int32'
              , 'old_task_id': 'VARCHAR'
              , 'task_name': 'VARCHAR'
              , 'max_chance': 'int32'
              , 'value_one_chance': 'int32'
              , 'mechanics': 'VARCHAR'
              , 'priority': 'int32'
              , 'action_id': 'int32'
              , 'code_tc': 'VARCHAR'
              , 'period_type': 'VARCHAR'
              , 'type_mechanics': 'VARCHAR'
              , 'status_field"': 'VARCHAR'
               }
# запрос
df_task = ddb.sql(f"""
   SELECT *
   FROM read_csv('{raw_data_file}',
   delim = ';',
   quote = '"',
   header = true,
   columns = {file_schema}
   )
""")

* period

In [45]:
# исходный файл
raw_data_file = raw_data_path + '\\' + 'lk_chance_data_period.csv'
# схема
file_schema = {'start_date': 'DATE'
              ,'stop_date': 'DATE'
              ,'period_id': 'int32'
               }
# запрос
df_period = ddb.sql(f"""
   SELECT *
   FROM read_csv('{raw_data_file}',
   delim = ';',
   quote = '"',
   header = true,
   columns = {file_schema}
   )
""")

* task_period

In [46]:
# исходный файл
raw_data_file = raw_data_path + '\\' + 'lk_chance_data_task_period.csv'
# схема
file_schema = {'dataflow_dttm': 'date'
              ,'period_id': 'int32'
              ,'task_id': 'int32'
               }
# запрос
df_task_period = ddb.sql(f"""
   SELECT *
   FROM read_csv('{raw_data_file}',
   delim = ';',
   quote = '"',
   header = true,
   columns = {file_schema}
   )
""")

* id данные

In [47]:
# исходный файл
raw_data_file = raw_data_path + '\\' + 'lk_chance_data_x5id.csv'
# схема
file_schema = {'x5id': 'VARCHAR'
              ,'user_x5_id': 'VARCHAR'
              ,'mobile_phone_no': 'VARCHAR'
              ,'mail_txt': 'VARCHAR'
              ,'advertising_x5id_flg': 'int16'
               }
# запрос
df_x5id = ddb.sql(f"""
   SELECT *
   FROM read_csv('{raw_data_file}',
   delim = ';',
   quote = '"',
   header = true,
   columns = {file_schema}
   )
""")

In [58]:
ddb.sql(f"""
   with 
   -- отбираем id из таблицы с заданиями
   t_task_periods_id as
       (
        select
           distinct
           tp.period_id
       from 
           df_task_period tp
       join
           (select task_id from df_task) ti
               on tp.task_id = ti.task_id
       )
   -- фильтруем таблицу периодов
       select 
           p.start_date
           , p.stop_date
           , p.period_id
       from
           df_period p
       join
           t_task_periods_id pi
               on p.period_id = pi.period_id
       order by start_date
""").to_df().to_clipboard(sep=';', index = False)

### Формирование отчетности

In [48]:
df_gen_dates = ddb.sql(f"""
   with 
   -- отбираем id из таблицы с заданиями
   t_task_periods_id as
       (
       select
           distinct
           tp.period_id
       from 
           df_task_period tp
       join
           (select task_id from df_task) ti
               on tp.task_id = ti.task_id
       )
   -- фильтруем таблицу периодов
   , t_periods as (
       select 
           p.start_date
           , p.stop_date
           , p.period_id
       from
           df_period p
       join
           t_task_periods_id pi
               on p.period_id = pi.period_id
   )
   -- генерируем даты
   , t_dates as 
       (
       SELECT
           CAST(generate_series AS DATE) AS report_date
       FROM
           generate_series(
               (select min(start_date) from t_periods)
               , (select max(stop_date) from t_periods)
               , INTERVAL '1 day'
               )
       )
   -- отчет
   select
       td.report_date
       , p.period_id
   from 
       t_dates td
   join
       t_periods p
           on td.report_date between p.start_date and p.stop_date
""")

In [49]:
df_participant_w_periods = ddb.sql(f"""
   SELECT
       p.x5id
       , p.grant_date_m as registration_date
       , p.nickname
       , gp.report_date
       , gp.period_id
   FROM 
       df_participant p
   CROSS JOIN
       df_gen_dates gp
""")

In [50]:
df_participant_progress_proc = ddb.sql(
f"""
   with t_main as (
       SELECT
           distinct
           pp.period_id
           , pp.item_date
           , pp.x5id
           , pp.task_id
           , t.code_tc
           , count(pp.trace_id) over w1 as chances_on_date_on_task
           , (count(pp.trace_id) over (partition by pp.x5id order by pp.item_date) > 1)::int as is_more_1_chance
           , 0 as is_completed
           , count(distinct pp.x5id) over (partition by pp.period_id, pp.item_date, t.code_tc) as cnt_users_by_tc_in_day
           , count(distinct pp.x5id) over (partition by pp.period_id, pp.item_date) as cnt_users_in_day
           , count(distinct pp.task_id) over (partition by pp.period_id, pp.item_date, pp.x5id, t.code_tc) as cnt_task_by_tc_in_day
           , count(distinct pp.task_id) over (partition by pp.period_id, pp.item_date, pp.x5id) as cnt_task_in_day
           , count(distinct t.code_tc) over (partition by pp.period_id, pp.item_date, pp.x5id) as cnt_tc_in_day
       FROM
           df_participant_progress pp
       JOIN
           df_task t
               on pp.task_id = t.task_id
       WINDOW w1 as (partition by pp.period_id, pp.item_date, pp.x5id, pp.task_id)
   )
   select
       t.*
       , sum(chances_on_date_on_task) over (partition by period_id, item_date, x5id) as chances_on_date_on_user
   from
       t_main t
   order by
       x5id, item_date, task_id
""")

<div class="alert alert-block alert-danger">
<b>Проверка:</b> Добавить проверку по id периода в исходнике и по справочнику на основании даты чека (коммент от 09.11.2025)
</div>

In [51]:
df_chances_report = ddb.sql(f"""
   with t_main as (
       SELECT
           p.x5id
           , p.report_date
           , (p.report_date=p.registration_date)::int as is_registration
           , p.nickname
           , p.period_id
           --, pp.period_id
           , prd.start_date
           , prd.stop_date
           , pp.item_date
           , pp.task_id
           , t.task_name
           , t.code_tc
           , chances_on_date_on_task
           , chances_on_date_on_user
           , is_completed
           , cnt_users_by_tc_in_day
           , cnt_users_in_day
           , cnt_task_by_tc_in_day
           , cnt_task_in_day
           , cnt_tc_in_day
           , is_more_1_chance
           , mobile_phone_no
           , mail_txt
           , advertising_x5id_flg
       FROM 
           df_participant_w_periods p
       LEFT JOIN
           df_participant_progress_proc pp
               on p.x5id = pp.x5id
               and p.report_date = pp.item_date
       LEFT JOIN
           df_x5id idx
               on p.x5id = idx.x5id
       JOIN
           df_period prd
               on p.period_id = prd.period_id
       LEFT JOIN
           df_task t
               on pp.task_id = t.task_id
   )
   SELECT
       t.*
       , row_number() over (partition by period_id, report_date, x5id order by task_id) as rn
   FROM
       t_main t
   
   WHERE 
       (is_registration = 1 or item_date is not null)
   ORDER BY
       report_date, x5id, task_id
""")

In [53]:
# смотрим количество строк
ddb.sql(f"""
   SELECT
       count(*)
   FROM 
       df_chances_report
""")

┌──────────────┐
│ count_star() │
│   int64    │
├──────────────┤
│       94757 │
└──────────────┘

In [55]:
df_chances_report.to_df().to_clipboard(sep=';', index = False)

Выгрузка id:
1) Выгружаем и добавляем в файл r"\\vdi-dpro-fsr006.x5.ru\FolderRedirection-6\aleksan.zamuruev\Desktop\x5_work\Прочее\mondelez_idx_week_2.csv" и меняем неделю через книгу data_loader_nb
2) Обрабатываем скриптом ниже и проверяем, что нет дубликатов
3) Загружаем в отчет при формировании
4) По победителям проверяем, что нет пустых

In [14]:
# id клиентов
df_idx_unique = ddb.sql(f"""
   SELECT
       distinct x5id
   FROM 
       df_chances_report
""")

In [15]:
df_idx_unique.to_df().to_clipboard(sep=';', index = False)